# Example 1: Constructing the reduced Hessian

In this example, we will solve a model and check the eigenvalues of the reduced Hessian at the solution. The purpose of this example is to (a) practice using `PyomoNLP` and (b) introduce you to the reduced Hessian, which can be useful for debugging regularization coefficients.

We start with some imports.

In [1]:
import pyomo.environ as pyo
import pyomo.environ as pyo
from pyomo.common.collections import ComponentSet
from pyomo.contrib.incidence_analysis import IncidenceGraphInterface
from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
from svi.auto_thermal_reformer.fullspace_flowsheet import make_optimization_model
import numpy as np
import scipy as sp

The model we will solve comes from the "surrogate-vs-implicit" repository, available [here](https://github.com/robbybp/surrogate-vs-implicit). We have imported this package above, and will construct an autothermal reforming flowsheet model using its function `make_optimization_model`.

In [5]:
def make_and_solve_model(**kwds):
    # First, we make our model
    P = kwds.pop("P", 1.5e6)
    # With X = 0.95, we converge. With X = 0.94, we don't. Go figure
    X = kwds.pop("X", 0.94)
    m = make_optimization_model(X, P)
    m.ipopt_zL_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.ipopt_zU_out = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)
    # Does the model solve?
    solver = pyo.SolverFactory("cyipopt", options=kwds)
    res = solver.solve(m, tee=True)
    # Converges infeasible in 900 iter
    return m, res

In [7]:
m, res = make_and_solve_model(X=0.95)
pyo.assert_optimal_termination(res)

2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer.control_volume.properties_in: Starting initialization
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer.control_volume.properties_out: Starting initialization
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer.control_volume.properties_out: Property initialization: optimal - Optimal Solution Found.
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer.control_volume: Initialization Complete
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer: Initialization Complete: optimal - Optimal Solution Found
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer_recuperator.hot_side.properties_in: Starting initialization
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer_recuperator.hot_side.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-05-06 21:44:53 [INFO] idaes.init.fs.reformer_recupera

Now we'll construct the reduced Hessian at the solution. This will give us access to the derivative matrices we'll need to construct the reduced Hessian.

Now, we'll construct the `PyomoNLP` object at the solution. This will give us access to the derivative matrices we need for the reduced Hessian.